## Imports

In [1]:
import pandas as pd
import numpy as np
import ast
import json

## Loading in the Data

In [2]:
powers_df = pd.read_csv('Data/superhero_powers - superhero_powers.csv')
powers_df.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


In [3]:
info_df = pd.read_csv('Data/superhero_info - superhero_info.csv')
info_df.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


## Cleaning Superhero Info Data

I will first start with the dataset for the superhero information. It looks like the columns that will need transforming are Hero|Publisher and measurements.

### Splitting the "Hero|Publisher" column

In [4]:
# first to explore the existing format
info_df['Hero|Publisher'].head(2)

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
Name: Hero|Publisher, dtype: object

The column separates the information with "|", so that will be used to separate the hero and publisher.

In [5]:
# adding expand=True
info_df['Hero|Publisher'].str.split('|',expand=True)

,0,1
0,A-Bomb,Marvel Comics
1,Abe Sapien,Dark Horse Comics
2,Abin Sur,DC Comics
3,Abomination,Marvel Comics
4,Absorbing Man,Marvel Comics
...,...,...
458,Yellowjacket,Marvel Comics
459,Yellowjacket II,Marvel Comics
460,Yoda,George Lucas
461,Zatanna,DC Comics


Now that they have been split, the columns can be renamed and saved. From there, they will be inserted back into the dataframe.

In [6]:
## save the 2 new columns into the dataframe
info_df[['Hero','Publisher']] = info_df['Hero|Publisher'].str.split('|',
                                                                    expand=True)
info_df.head()


,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


Now that the columns have been added, I can drop the original column.

In [7]:
## drop the original column 
info_df = info_df.drop(columns='Hero|Publisher')
info_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics


### Splitting the Measurements Column

In [8]:
## examining a single value from the Measurements col
meas = info_df.loc[0,"Measurements"]
print(type(meas))
meas

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

The dictionary is loaded as a string, so the dictionary will not be able to be split until the quotations are removed from the outside and the quotes inside are changed to double quotes.

In [9]:
meas = meas.replace("'",'"')
meas

'{"Height": "203.0 cm", "Weight": "441.0 kg"}'

In [10]:
# now I can use json.loads
fixed_meas = json.loads(meas)
print(type(fixed_meas))
fixed_meas

<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

Now that I know this worked, I can apply it to the entire column.

In [11]:
# use .str.replace to replace all single quotes
info_df['Measurements'] = info_df['Measurements'].str.replace("'",'"')
# Apply the json.loads to the full column
info_df['Measurements'] = info_df['Measurements'].apply(json.loads)
info_df['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [12]:
# check a single value after transformation
test_meas = info_df.loc[0, 'Measurements']
print(type(test_meas))
test_meas


<class 'dict'>


{'Height': '203.0 cm', 'Weight': '441.0 kg'}

Now that the column has been made into a dictionary, I can separate it into 2 columns.

In [13]:
height_weight = info_df['Measurements'].apply(pd.Series)
height_weight

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


Even though the dictionary was successfully split, I want to remove the 'cm' and 'kg' from the values.

In [14]:
# Make a list of all characters to replace
to_replace = ['cm']
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    height_weight['Height'] = height_weight['Height'].str.replace(char,'',regex=False)
    
height_weight['Height'].head()

0    203.0 
1    191.0 
2    185.0 
3    203.0 
4    193.0 
Name: Height, dtype: object

Now I will do the same for the 'Weight' column.

In [15]:
# Make a list of all characters to replace
to_replace = ['kg']
# run a loop to replace all of the characters in the list at once
for char in to_replace:
    height_weight['Weight'] = height_weight['Weight'].str.replace(char,'',regex=False)
    
height_weight['Weight'].head()

0    441.0 
1     65.0 
2     90.0 
3    441.0 
4    122.0 
Name: Weight, dtype: object

I want to add the 'cm' and 'kg' back into the column name, so I will do that before adding the columns back into the dataframe.

In [16]:
height_weight = height_weight.rename(columns={'Height': 'Height (cm)', 
                                              'Weight': 'Weight (kg)'})
height_weight.head()

,Height (cm),Weight (kg)
0,203.0,441.0
1,191.0,65.0
2,185.0,90.0
3,203.0,441.0
4,193.0,122.0


Now I want to combine the new dataframe with the info_df.

In [17]:
# concat height_weight with original dataframe
info_df = pd.concat((info_df, height_weight), axis = 1)
info_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height (cm),Weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}",Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}",Absorbing Man,Marvel Comics,193.0,122.0


In [18]:
# drop the original 'Measurements' column
info_df = info_df.drop(columns='Measurements')
info_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (cm),Weight (kg)
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0


In [27]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Gender       463 non-null    object
 1   Race         463 non-null    object
 2   Alignment    463 non-null    object
 3   Hair color   463 non-null    object
 4   Eye color    463 non-null    object
 5   Skin color   463 non-null    object
 6   Hero         463 non-null    object
 7   Publisher    463 non-null    object
 8   Height (cm)  463 non-null    object
 9   Weight (kg)  463 non-null    object
dtypes: object(10)
memory usage: 36.3+ KB


In [28]:
info_df['Height (cm)'] = info_df['Height (cm)'].astype(float)
info_df['Weight (kg)'] = info_df['Weight (kg)'].astype(float)

In [29]:
info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Gender       463 non-null    object 
 1   Race         463 non-null    object 
 2   Alignment    463 non-null    object 
 3   Hair color   463 non-null    object 
 4   Eye color    463 non-null    object 
 5   Skin color   463 non-null    object 
 6   Hero         463 non-null    object 
 7   Publisher    463 non-null    object 
 8   Height (cm)  463 non-null    float64
 9   Weight (kg)  463 non-null    float64
dtypes: float64(2), object(8)
memory usage: 36.3+ KB


This dataset looks good to go, so now I will move on to the superpowers dataset.

## Cleaning Superpower Data

The column I will address in this dataset will be the 'Powers' column.

In [19]:
## showing the lists are really strings
powers_df.loc[2,'Powers']

'Agility,Accelerated Healing,Cold Resistance,Durability,Underwater breathing,Marksmanship,Weapons Master,Longevity,Intelligence,Super Strength,Telepathy,Stamina,Immortality,Reflexes,Enhanced Sight,Sub-Mariner'

In [20]:
powers_split = powers_df['Powers'].str.split(',')
powers_split

0        [Agility, Super Strength, Stamina, Super Speed]
1      [Accelerated Healing, Durability, Longevity, S...
2      [Agility, Accelerated Healing, Cold Resistance...
3                                   [Lantern Power Ring]
4      [Accelerated Healing, Intelligence, Super Stre...
                             ...                        
662               [Flight, Energy Blasts, Size Changing]
663    [Cold Resistance, Durability, Longevity, Super...
664    [Agility, Stealth, Danger Sense, Marksmanship,...
665    [Cryokinesis, Telepathy, Magic, Fire Control, ...
666    [Super Speed, Intangibility, Time Travel, Time...
Name: Powers, Length: 667, dtype: object

Now I want to make new columns with the values in order to one hot encode.

In [24]:
cols_to_make = powers_split.explode('Powers').dropna().sort_values().unique()
cols_to_make

array(['Accelerated Healing', 'Adaptation', 'Agility',
       'Animal Attributes', 'Animal Control', 'Animal Oriented Powers',
       'Animation', 'Anti-Gravity', 'Astral Projection', 'Astral Travel',
       'Audio Control', 'Banish', 'Biokinesis', 'Camouflage',
       'Changing Armor', 'Clairvoyance', 'Cloaking', 'Cold Resistance',
       'Cryokinesis', 'Danger Sense', 'Darkforce Manipulation',
       'Death Touch', 'Density Control', 'Dexterity',
       'Dimensional Awareness', 'Dimensional Travel', 'Duplication',
       'Durability', 'Echolocation', 'Elasticity', 'Electrical Transport',
       'Electrokinesis', 'Element Control',
       'Elemental Transmogrification', 'Empathy', 'Energy Absorption',
       'Energy Armor', 'Energy Beams', 'Energy Blasts',
       'Energy Constructs', 'Energy Manipulation', 'Energy Resistance',
       'Enhanced Hearing', 'Enhanced Memory', 'Enhanced Senses',
       'Enhanced Sight', 'Enhanced Smell', 'Enhanced Touch',
       'Fire Control', 'Fire Resis

In [25]:
for col in cols_to_make:
    powers_df[col] = powers_df['Powers'].str.contains(col)
    
powers_df.head()

C:\Users\joshl\AppData\Local\Temp\ipykernel_13780\4127372729.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers_df[col] = powers_df['Powers'].str.contains(col)
C:\Users\joshl\AppData\Local\Temp\ipykernel_13780\4127372729.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  powers_df[col] = powers_df['Powers'].str.contains(col)
C:\Users\joshl\AppData\Local\Temp\ipykernel_13780\4127372729.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poo

,hero_names,Powers,Accelerated Healing,Adaptation,Agility,Animal Attributes,Animal Control,Animal Oriented Powers,Animation,Anti-Gravity,...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed",False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super...",True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du...",True,False,True,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,Abin Sur,Lantern Power Ring,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt...",True,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False


In [26]:
# drop 'Powers' column
powers_df = powers_df.drop(columns='Powers')
powers_df.head()

,hero_names,Accelerated Healing,Adaptation,Agility,Animal Attributes,Animal Control,Animal Oriented Powers,Animation,Anti-Gravity,Astral Projection,...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,3-D Man,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
3,Abin Sur,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


## Merge DataFrames

In [30]:
# merging both dataframes
final_df = pd.merge(info_df, powers_df, left_on='Hero', right_on='hero_names', how='inner')
final_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (cm),Weight (kg),...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,True,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


In [31]:
final_df = final_df.drop(columns='hero_names')
final_df.head()

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height (cm),Weight (kg),...,Vision - Thermal,Vision - X-Ray,Vitakinesis,Wallcrawling,Water Control,Weapon-based Powers,Weapons Master,Weather Control,Web Creation,Wind Control
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,...,False,False,False,False,False,False,True,False,False,False
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,...,False,False,False,False,False,False,False,False,False,False
3,Male,Human / Radiation,bad,No Hair,green,Unknown,Abomination,Marvel Comics,203.0,441.0,...,False,False,False,False,False,False,False,False,False,False
4,Male,Human,bad,No Hair,blue,Unknown,Absorbing Man,Marvel Comics,193.0,122.0,...,False,False,False,False,False,False,False,False,False,False


In [37]:
super_speed = final_df.groupby(['Super Speed'])['Weight (kg)'].mean()
super_speed

Super Speed
False    101.773585
True     129.404040
Name: Weight (kg), dtype: float64

In [40]:
hero_height = final_df.groupby(['Publisher'])['Height (cm)'].mean().sort_values(ascending=False)
hero_height

Publisher
Image Comics         211.000000
Marvel Comics        191.546128
DC Comics            181.923913
Star Trek            181.500000
Team Epic TV         180.750000
Unknown              178.000000
Dark Horse Comics    176.909091
Shueisha             171.500000
George Lucas         159.600000
Name: Height (cm), dtype: float64